# Open AI modelių tobulinimas

Šis užrašų knygelės pavyzdys remiasi dabartinėmis gairėmis, pateiktomis Open AI [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) dokumentacijoje.

Tobulinimas pagerina pagrindinių modelių veikimą jūsų programoje, perkvalifikuojant jį su papildomais duomenimis ir kontekstu, susijusiu su konkrečia naudojimo sritimi ar scenarijumi. Atkreipkite dėmesį, kad užklausų inžinerijos metodai, tokie kaip _few shot learning_ ir _retrieval augmented generation_, leidžia pagerinti numatytą užklausą su susijusiais duomenimis, kad pagerintų kokybę. Tačiau šie metodai yra ribojami tikslinio pagrindinio modelio maksimalaus žodžių lango dydžio.

Tobulinant modelį, mes efektyviai perkvalifikuojame patį modelį su reikiamais duomenimis (leidžiant naudoti daug daugiau pavyzdžių nei telpa maksimaliame žodžių lange) – ir diegiame _individualią_ modelio versiją, kuriai nebereikia pateikti pavyzdžių vykdymo metu. Tai ne tik pagerina mūsų užklausos dizaino efektyvumą (turime daugiau lankstumo naudoti žodžių langą kitoms užduotims), bet ir potencialiai sumažina mūsų išlaidas (mažinant žodžių, kuriuos reikia siųsti modeliui vykdymo metu, skaičių).

Tobulinimas turi 4 žingsnius:
1. Paruošti mokymo duomenis ir juos įkelti.
1. Paleisti mokymo užduotį, kad gautumėte tobulintą modelį.
1. Įvertinti tobulintą modelį ir kartoti, siekiant kokybės.
1. Diegti tobulintą modelį vykdymui, kai būsite patenkinti.

Atkreipkite dėmesį, kad ne visi pagrindiniai modeliai palaiko tobulinimą – [patikrinkite OpenAI dokumentaciją](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) dėl naujausios informacijos. Taip pat galite tobulinti anksčiau tobulintą modelį. Šiame vadove naudosime `gpt-35-turbo` kaip mūsų tikslinį pagrindinį modelį tobulinimui.

---


### 1.1 žingsnis: Paruoškite savo duomenų rinkinį

Sukurkime pokalbių robotą, kuris padės jums suprasti elementų periodinę lentelę, atsakydamas į klausimus apie elementą limeriku. Šiame paprastame vadove mes tiesiog sukursime duomenų rinkinį, kad apmokytume modelį su keliais pavyzdiniais atsakymų pavyzdžiais, kurie parodo duomenų formatą. Tikroje situacijoje reikėtų sukurti daug daugiau pavyzdžių turintį duomenų rinkinį. Taip pat galite naudoti atvirą duomenų rinkinį (jūsų taikymo sričiai), jei toks egzistuoja, ir jį pertvarkyti, kad būtų galima naudoti smulkiam apmokymui.

Kadangi mes orientuojamės į `gpt-35-turbo` ir ieškome vieno atsakymo (pokalbio užbaigimo), galime sukurti pavyzdžius naudodami [šią siūlomą formatą](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), atitinkantį OpenAI pokalbio užbaigimo reikalavimus. Jei tikitės daugiažingsnio pokalbio turinio, naudotumėte [daugiažingsnio pavyzdžio formatą](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), kuris apima `weight` parametrą, nurodantį, kurie pranešimai turėtų būti naudojami (ar ne) smulkiam apmokymui.

Šiame vadove naudosime paprastesnį vieno atsakymo formatą. Duomenys pateikti [jsonl formatu](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) su 1 įrašu kiekvienoje eilutėje, kiekvienas pateiktas kaip JSON objektas. Žemiau pateiktas fragmentas rodo 2 įrašus kaip pavyzdį – žr. [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) pilną pavyzdžių rinkinį (10 pavyzdžių), kurį naudosime smulkiam apmokymui. **Pastaba:** Kiekvienas įrašas _privalo_ būti aprašytas vienoje eilutėje (nepadalintas į kelias eilutes, kaip įprasta formatuotame JSON faile)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Tikroje situacijoje jums reikės daug didesnio pavyzdžių rinkinio geresniems rezultatams – kompromisas bus tarp atsakymų kokybės ir smulkaus apmokymo laiko/kainos. Mes naudojame mažą rinkinį, kad galėtume greitai užbaigti smulkų apmokymą ir iliustruoti procesą. Žr. [šį OpenAI Cookbook pavyzdį](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) sudėtingesniam smulkaus apmokymo vadovui.


---

### 1.2 žingsnis. Įkelkite savo duomenų rinkinį

Įkelkite duomenis naudodami Files API [kaip aprašyta čia](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Atkreipkite dėmesį, kad norint paleisti šį kodą, pirmiausia turite atlikti šiuos veiksmus:
 - Įdiegti `openai` Python paketą (įsitikinkite, kad naudojate versiją >=0.28.0, kad gautumėte naujausias funkcijas)
 - Nustatyti `OPENAI_API_KEY` aplinkos kintamąjį su savo OpenAI API raktu
Norėdami sužinoti daugiau, žr. [Nustatymo vadovą](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst), pateiktą kurse.

Dabar paleiskite kodą, kad sukurtumėte failą įkėlimui iš savo vietinio JSONL failo.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### 2.1 žingsnis: Sukurkite Fine-tuning užduotį naudodami SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### 2.2 žingsnis: Patikrinkite užduoties būseną

Štai keletas dalykų, kuriuos galite padaryti su `client.fine_tuning.jobs` API:
- `client.fine_tuning.jobs.list(limit=<n>)` - Išvardinti paskutines n tobulinimo užduočių
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Gauti konkrečios tobulinimo užduoties detales
- `client.fine_tuning.jobs.cancel(<job_id>)` - Atšaukti tobulinimo užduotį
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Išvardinti iki n įvykių iš užduoties
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Proceso pirmasis žingsnis yra _mokymo failo patvirtinimas_, kad įsitikintumėte, jog duomenys yra tinkamu formatu.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### 2.3 žingsnis: Stebėkite įvykius, kad galėtumėte sekti pažangą


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### 2.4 žingsnis: Peržiūrėkite būseną OpenAI informacijos suvestinėje


Taip pat galite peržiūrėti būseną apsilankę OpenAI svetainėje ir naršydami platformos _Fine-tuning_ skiltyje. Čia matysite dabartinio darbo būseną, taip pat galėsite sekti ankstesnių darbo vykdymo kartų istoriją. Šiame ekrano paveikslėlyje matote, kad ankstesnis vykdymas nepavyko, o antrasis bandymas buvo sėkmingas. Kontekste, tai įvyko, kai pirmasis bandymas naudojo JSON failą su neteisingai suformatuotais įrašais – pataisius, antrasis bandymas sėkmingai užbaigė darbą ir padarė modelį prieinamą naudoti.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.lt.png)


Taip pat galite peržiūrėti būsenos pranešimus ir metrikas slinkdami žemyn vizualinėje informacijos suvestinėje, kaip parodyta:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.lt.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.lt.png)|


---

### 3.1 žingsnis: Gauti ID ir išbandyti smulkiai paruoštą modelį kode


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### 3.2 žingsnis: Įkelkite ir išbandykite smulkiai paruoštą modelį Playground aplinkoje

Dabar galite išbandyti smulkiai paruoštą modelį dviem būdais. Pirmiausia, galite apsilankyti Playground ir naudoti Models išskleidžiamąjį meniu, kad pasirinktumėte savo naujai smulkiai paruoštą modelį iš pateiktų variantų. Kitas variantas – naudoti "Playground" parinktį, rodomą Fine-tuning skydelyje (žr. aukščiau esantį ekrano vaizdą), kuri paleidžia šį _palyginamąjį_ vaizdą, rodantį pagrindinio ir smulkiai paruošto modelių versijas šalia viena kitos greitam įvertinimui.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.lt.png)

Tiesiog užpildykite sistemos kontekstą, naudotą jūsų mokymo duomenyse, ir pateikite savo testinį klausimą. Pastebėsite, kad abi pusės atnaujinamos su identišku kontekstu ir klausimu. Paleiskite palyginimą ir pamatysite skirtumą tarp jų atsakymų. _Atkreipkite dėmesį, kaip smulkiai paruoštas modelis pateikia atsakymą tokiu formatu, kokį nurodėte savo pavyzdžiuose, o pagrindinis modelis tiesiog seka sistemos užklausą_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.lt.png)

Pastebėsite, kad palyginimas taip pat pateikia kiekvieno modelio žodžių (tokenų) skaičių ir laiką, užtruktą inferencijai. **Šis konkretus pavyzdys yra paprastas ir skirtas parodyti procesą, bet neatskleidžia tikro pasaulio duomenų rinkinio ar scenarijaus**. Galite pastebėti, kad abu pavyzdžiai rodo tą patį žodžių skaičių (sistemos kontekstas ir vartotojo užklausa yra identiški), o smulkiai paruoštas modelis užtrunka daugiau laiko inferencijai (individualus modelis).

Tikruose scenarijuose jūs nenaudosite tokio žaislinio pavyzdžio, o smulkiai paruošite modelį pagal tikrus duomenis (pvz., produktų katalogą klientų aptarnavimui), kur atsakymų kokybė bus daug labiau pastebima. _Tuo atveju, norint gauti lygiavertę atsakymų kokybę su pagrindiniu modeliu, reikės daugiau individualios užklausų inžinerijos, kuri padidins žodžių naudojimą ir galimai susijusį apdorojimo laiką inferencijai._ _Norėdami tai išbandyti, peržiūrėkite smulkiai paruošimo pavyzdžius OpenAI Cookbook._ 

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Atsakomybės apribojimas**:
Šis dokumentas buvo išverstas naudojant dirbtinio intelekto vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Svarbiai informacijai rekomenduojamas profesionalus žmogaus vertimas. Mes neatsakome už bet kokius nesusipratimus ar neteisingus aiškinimus, kilusius dėl šio vertimo naudojimo.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
